In [2]:
import os
import glob
import subprocess

import doralite
import matplotlib.pyplot as plt
import momgrid as mg
import momlevel as ml
import numpy as np
import xarray as xr

In [37]:
path = "/work/Katherine.Turner/tempest-extremes/CM4_cmip/data/"
path = "/archive/oar.gfdl.cmip6/ESM4/DECK/ESM4_piControl_D/gfdl.ncrc4-intel16-prod-openmp/pp/ocean_monthly_z_1x1deg/ts/monthly/5yr/"
# momgrid will use a directory of pre-computed weights
import momgrid
os.environ["MOMGRID_WEIGHTS_DIR"] = "/nbhome/John.Krasting/grid_weights"

In [40]:
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)

for i in range(0,50):
    ds_t = xr.open_mfdataset(f'{path}ocean_monthly_z_1x1deg.{i:03d}*.thetao.nc', decode_times=time_coder)
    ds_s = xr.open_mfdataset(f'{path}ocean_monthly_z_1x1deg.{i:03d}*.so.nc', decode_times=time_coder)
    
    ds_t = ds_t.sel(lat=slice(-90,-30))
    ds_s = ds_s.sel(lat=slice(-90,-30))

    zeta = 0.0
    coriolis = ml.derived.calc_coriolis(ds_t.lat)
    n2 = ml.derived.calc_n2(ds_t.thetao, ds_s.so)
    pv = ml.derived.calc_pv(zeta, coriolis, n2, interp_n2=False, units="cm")

    pv = pv.transpose('time', 'z_l', 'lat', 'lon').compute()

    pv.to_dataset(name='pv').to_netcdf(f'/work/Katherine.Turner/tempest-extremes/ESM4_cmip/data/ocean_monthly_z_1x1deg.0{i:03d}1-0{i+1:03d}0.pv.nc')